### mysql과 python을 연동 
1. mysql과 연동할 수 있는 기능을 가진 라이브러리 설치 
    - 라이브러리명 : pymysql
    - pip install pymysql 
2. mysql 서버의 정보를 입력하고 연결 
    - host(서버의 주소)
        - 127.0.0.1, localhost (내 컴퓨터)
        - 외부의 주소 : 172.168.1.40
    - port(컴퓨터의 보안)
        - mysql의 기본 port : 3306
    - user(mysql 연결시 사용하는 id)
        - mysql의 기본 user : root
    - password (mysql 연결시 사용하는 비밀번호)
        - root의 비밀번호 : 1234
        - 비밀번호가 존재하지 않으면 생략 가능
    - db (Database의 이름)
        - mysql에서 사용할 데이터베이스의 이름
3. DB 서버와 python 공간 사이에 가상 공간을 생성
    - Cursor라는 가상 공간 생성
4. sql 쿼리문을 이용하여 cursor에 질의를 보낸다. 
    - 데이터의 변화가 존재하는 쿼리(insert, update, delete)
        - 질의를 보낸다 ( execute() )
        - DB 서버의 데이터를 변경 ( commit() )
        - DB 서버와의 연결을 종료 ( close() )
    - 데이터를 조회하는 쿼리 (select)
        - 질의를 보낸다 ( execute() )
        - 질의에 대한 결과 값을 받아온다. ( fetchall() )
            - cursor에서 데이터를 받아오면 cursor에 조회된 데이터는 초기화
        - DB 서버와의 연결을 종료 ( close() )

In [ ]:
# 라이브러리 설치 
# !pip install pymysql

In [2]:
# 라이브러리 로드 
import pymysql

In [ ]:
# mysql 연결시 에러 발생 시 추가적인 라이브러리 설치 
!pip install cryptography

In [9]:
# mysql 서버의 정보를 입력하고 연결 
_db = pymysql.connect(
    host = 'localhost', 
    port = 3306, 
    user = 'root', 
    password=  '1234', 
    db = 'ezen'
)

In [10]:
# 가상 공간을 생성 (Cursor)
# 기본값으로 가상 공간 생성 -> select 질의에 대한 결과 값이 tuple 형태로 되돌려주는 가상 공간
cursor = _db.cursor()
# select 질의에 대한 결과 값이 dict 형태로 되돌려주는 가상 공간 생성
cursor2 = _db.cursor(pymysql.cursors.DictCursor)

In [5]:
# sql 쿼리문을 작성 
# user table의 모든 열과 모든 행을 조회
select_query = "select * from user"

# cursor에 질의의 보낸다 
cursor.execute(select_query)
cursor2.execute(select_query)

1

In [6]:
# cursor에서 결과 값을 되돌려받는다. 
cursor.fetchall()

(('test', '1234', 'kim', None),)

In [7]:
cursor2.fetchall()

[{'id': 'test', 'pass': '1234', 'name': 'kim', 'phone': None}]

In [8]:
# DB 서버와의 연결을 종료
_db.close()

In [11]:
# 데이터의 변화가 존재하는 쿼리문을 생성 
insert_query = """
    insert into 
    `user`(`id`, `pass`)
    values ('test3', '0000')
"""
# cursor에 쿼리문을 보낸다. 
# DB 서버는 아직 데이터가 변화하지 않는다. 
cursor2.execute(insert_query)

1

In [12]:
select_query2 = """
    select * from user
"""
cursor2.execute(select_query2)
print(cursor2.fetchall())

[{'id': 'test', 'pass': '1234', 'name': 'kim', 'phone': None}, {'id': 'test3', 'pass': '0000', 'name': None, 'phone': None}]


In [13]:
# cursor에서 변경된 데이터를 DB 서버에 동기화
_db.commit()

In [14]:
_db2 = pymysql.connect(
    host = 'moons-86.iptime.org', 
    port = 3306, 
    user = 'ezen', 
    password = '1234', 
    db = 'ezen'
)

In [16]:
# 쿼리문을 생성 + 쿼리문에 들어가는 데이터를 따로 설정 
# 내 컴퓨터에 있는 DB server에서 특정 아이디와 비밀번호를 조회 하는 쿼리문 생성 
login_query = """
    select * from user 
    where 
    id = 'test' and pass = '1111'
"""
cursor2.execute(login_query)
cursor2.fetchall()

()

In [18]:
# 쿼리문과 데이터를 나눠서 변수에 저장 
login_query2 = """
    select * from user 
    where 
    id = %s and pass = %s
"""
input_id = input("아이디를 입력하시오")
input_pass = input('패스워드를 입력하시오')
print(input_id, input_pass)

# login_query2에 input_id, input_pass를 대입 -> execute() 자동으로 대입 
# execute( 쿼리문, [쿼리문의 %s에 들어갈 데이터] )
cursor2.execute(login_query2, [input_id, input_pass])
print(cursor2.fetchall())

test 0000
()


In [27]:
test_query = """
    SELECT * from user
"""
test_query2 = """
    insert into user values ('test4', '0000')
"""
# 문자열의 좌우의 공백을 제거(strip) -> 문자를 모두 소문자로 변경(lower) -> 문자열의 시작이 select인가(startswith)
print(test_query.strip().lower().startswith('select'))
print(test_query2.strip().lower().startswith('select'))
# 문자열을 소문자로 변경(lower) -> 문자열을 특정 문자를 기준으로 나눠준다(split())
print(test_query.lower().split()[0] == 'select')
print(test_query2.lower().split()[0] =='select')

False
False
True
False


In [28]:
# 함수 생성 
# 매개변수 2개 : 쿼리문, *데이터
def query_execute(_query, *_values):
    # 질의를 cursor에 보낸다.(execute)
    # 만약에 query문이 잘못 되었을때(에러 발생시)
    try:
        cursor2.execute(_query, _values)
    except:
        return "쿼리문 에러 발생"
    # _query가 select문이라면?
    if _query.strip().lower().startswith('select'):
        # cursor에서 결과값을 되돌려 받는다. 
        result = cursor2.fetchall()
    # _query가 select이 아니라면?
    else:
        # DB server에 cursor를 동기화
        _db.commit()
        result = 'Query OK'
    return result

In [29]:
# 함수 호출 (쿼리문만 인자값으로 호출)
# 쿼리문 작성 
sql_query = """
    select * from user
"""
query_execute(sql_query)

[{'id': 'test', 'pass': '1234', 'name': 'kim', 'phone': None},
 {'id': 'test3', 'pass': '0000', 'name': None, 'phone': None}]

In [30]:
# 함수 호출 (쿼리문(select) + 데이터)
sql_query2 = """
    SELECT * from user 
    WHERE id = %s
"""
query_execute(sql_query2, 'test3')

[{'id': 'test3', 'pass': '0000', 'name': None, 'phone': None}]

In [31]:
# 함수 호출 (쿼리문(update) + 데이터)
sql_query3 = """
    update user set pass = %s 
    where id = %s
"""
query_execute(sql_query3, '0123', 'test3')

'Query OK'

### DB 서버와의 연결을 class 화
1. 생성자 함수 
    - class를 생성할때 class안에서 독립적으로 사용하는 변수의 데이터를 대입을 하는 과정 
    - DB server의 정보를 독립 변수에 대입 
        - host, port, user, password, db
2. sql query문과 데이터를 입력받아서 질의를 보내고 응답 메시지를 되돌려 받는 함수 생성 
    - 생성자 함수를 통해서 생성한 DB server의 정보를 가지고 DB server와 연결 
    - 가상 공간인 cursor를 생성 
    - execute()함수를 이용하여 질의의 보낸다. 
    - 쿼리문이 select문이라면 가상 공간에서 결과를 되돌려받아서 변수(result)에 대입
    - select문이 아니라면 DB server에 가상 공간 데이터를 동기화
    - result 변수를 생성하여 'Query OK' 문자열을 대입
    - DB server와의 연결을 종료
    - result를 되돌려준다. 


In [33]:
# class 선언 
class MyDB:
    # 생성자 함수 -> 매개변수의 기본값은 내 컴퓨터로 지정
    def __init__(
        self,
        _host = 'localhost', 
        _port = 3306, 
        _user = 'root', 
        _pw = '1234', 
        _db = 'ezen'
    ):
        # class 내부에서 사용이 되는 독립 변수를 생성 
        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db = _db
    
    # class 안에서 사용을 하는 내장 함수 생성 
    def query_execute(self, _query, *_values):
        # DB server와 연결 
        _db = pymysql.connect(
            host = self.host, 
            port = self.port, 
            user = self.user, 
            password = self.pw, 
            db = self.db
        )
        # 가상 공간(cursor를 생성)
        cursor = _db.cursor(pymysql.cursors.dictCursor)
        try:
            # 입력받은 query문을 cursor에 질의를 보낸다. 
            cursor.execute(_query, _values)
        except:
            return "쿼리문 에러 발생"
        # select문인 경우 
        if _query.strip().upper().startswith('SELECT'):
            # cursor에서 결과값을 되돌려받는다. 
            result = cursor.fetchall()
        # select문이 아닌 경우 
            # DB server에 가상 공간 데이터를 동기화
            _db.commit()
            result = 'Query OK'
        # DB server와의 연결을 종료
        _db.close()
        # result를 되돌려준다. 
        return result

In [34]:
# class 생성 (내 컴퓨터에 접속)
db1 = MyDB()

In [35]:
# class 생성 ( 외부의 데이터 서버에 접속 )
db2 = MyDB(
    _host = 'moons-86.iptime.org', 
    _user = 'ezen', 
    _pw = '1234', 
    _db = 'ezen'
)